In [1]:
from mitools import reg
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.datasets.longley import load_pandas
from mitools.visuals import LinePlotter

In [2]:
np.random.seed(9876789)

# OLS Regression from Statsmodels

https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html

## OLS estimation

#### Artificial data:

In [3]:
nsample = 100
x = np.linspace(0, 10, 100)
X = np.column_stack((x, x ** 2))
beta = np.array([1, 0.1, 10])
e = np.random.normal(size=nsample)

#### Our model needs an intercept so we add a column of 1s:

In [4]:
X = sm.add_constant(X)
y = np.dot(X, beta) + e

#### Fit and summary:

In [ ]:
model = reg.OLSModel.from_arrays(y, X)
results = model.fit()
print(results.summary())

Quantities of interest can be extracted directly from the fitted model. Type `dir(results)` for a full list. Here are some examples:

In [ ]:
print("Parameters: ", results.params)
print("R2: ", results.rsquared)


## OLS non-linear curve but linear in parameters

#### We simulate artificial data with a non-linear relationship between x and y:

In [7]:
nsample = 50
sig = 0.5
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, np.sin(x), (x - 5) ** 2, np.ones(nsample)))
beta = [0.5, 0.5, -0.02, 5.0]

y_true = np.dot(X, beta)
y = y_true + sig * np.random.normal(size=nsample)

#### Fit and summary:

In [ ]:
model = reg.OLSModel.from_arrays(y, X)
res = model.fit()
print(res.summary())

#### Extract other quantities of interest:

In [ ]:
print("Parameters: ", res.params)
print("Standard errors: ", res.bse)
print("Predicted values: ", res.predict())

Draw a plot to compare the true relationship to OLS predictions. Confidence intervals around the predictions are built using the `wls_prediction_std` command.


In [ ]:
pred_ols = res.get_prediction()
iv_l = pred_ols.summary_frame()["obs_ci_lower"]
iv_u = pred_ols.summary_frame()["obs_ci_upper"]

line_plot = (LinePlotter(x_data=[x, x, x, x, x], y_data=[y, y_true, res.fittedvalues, iv_u, iv_l])
.set_linestyle(['', '-', '-.', '--', '--'])
.set_marker(['o', None, None, None, None])
.set_color(['blue', 'blue', 'red', 'red', 'red'])
.set_label(['data', 'True', 'OLS', "", ""])
)
ax = line_plot.draw()
ax.legend()

## OLS with dummy variables

We generate some artificial data. There are 3 groups which will be modelled using dummy variables. Group 0 is the omitted/benchmark category.

In [11]:
nsample = 50
groups = np.zeros(nsample, int)
groups[20:40] = 1
groups[40:] = 2

dummy = pd.get_dummies(groups).values
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, dummy[:, 1:]))
X = sm.add_constant(X, prepend=False)

beta = [1.0, 3, -3, 10]
y_true = np.dot(X, beta)
e = np.random.normal(size=nsample)
y = y_true + e

#### Inspect the data:

In [ ]:
print(X[:5, :])
print(y[:5])
print(groups)
print(dummy[:5, :])

#### Fit and summary:

In [ ]:
model = reg.OLSModel.from_arrays(y, X)
res2 = model.fit()
print(res2.summary())

#### Draw a plot to compare the true relationship to OLS predictions:

In [ ]:
pred_ols2 = res2.get_prediction()
iv_l = pred_ols2.summary_frame()["obs_ci_lower"]
iv_u = pred_ols2.summary_frame()["obs_ci_upper"]

line_plot = (LinePlotter(x_data=[x, x, x, x, x], y_data=[y, y_true, res2.fittedvalues, iv_u, iv_l])
.set_linestyle(['', '-', '-.', '--', '--'])
.set_marker(['o', None, None, None, None])
.set_color(['blue', 'blue', 'red', 'red', 'red'])
.set_label(['data', 'True', 'OLS', "", ""])
)
ax = line_plot.draw()
_ = ax.legend(loc='upper left')

## Joint Hypothesis test

### F test

We want to test the hypothesis that both coefficients on the dummy variables are equal to zero, that is, $R \times \beta = 0$. An F test leads us to strongly reject the null hypothesis of identical constant in the 3 groups:

In [ ]:
R = [[0, 1, 0, 0], [0, 0, 1, 0]]
print(np.array(R))
print(res2.f_test(R))

You can also use formula-like syntax to test hypotheses

In [ ]:
print(res2.f_test("x2 = x3 = 0"))

### Small group effects

If we generate artificial data with smaller group effects, the T test can no longer reject the Null hypothesis:

In [17]:
beta = [1.0, 0.3, -0.0, 10]
y_true = np.dot(X, beta)
y = y_true + np.random.normal(size=nsample)
model = reg.OLSModel.from_arrays(y, X)
res3 = model.fit()

In [ ]:
print(res3.f_test(R))

In [ ]:
print(res3.f_test("x2 = x3 = 0"))

### Multicollinearity

The Longley dataset is well known to have high multicollinearity. That is, the exogenous predictors are highly correlated. This is problematic because it can affect the stability of our coefficient estimates as we make minor changes to model specification.

In [20]:
y = load_pandas().endog.to_frame()
X = load_pandas().exog
X = sm.add_constant(X)

In [ ]:
data = pd.concat([y, X], axis=1)
data

#### Fit and summary:

In [ ]:
model = reg.OLSModel(dependent_variable="TOTEMP", data=data)
ols_results = model.fit()
print(ols_results.summary())

### Condition Number

One way to assess multicollinearity is to compute the condition number. Values over 20 are worrisome (see Greene 4.9). The first step is to normalize the independent variables to have unit length:

In [23]:
norm_x = X.values
for i, name in enumerate(X):
    if name == "const":
        continue
    norm_x[:, i] = X[name] / np.linalg.norm(X[name])
norm_xtx = np.dot(norm_x.T, norm_x)

Then, we take the square root of the ratio of the biggest to the smallest eigen values.



In [ ]:
eigs = np.linalg.eigvals(norm_xtx)
condition_number = np.sqrt(eigs.max() / eigs.min())
print(condition_number)

### Dropping an observation

Greene also points out that dropping a single observation can have a dramatic effect on the coefficient estimates:

In [ ]:
model = reg.OLSModel(dependent_variable="TOTEMP", data=data.iloc[:14])
ols_results2 = model.fit()
print(
    "Percentage change %4.2f%%\n"
    * 7
    % tuple(
        [
            i
            for i in (ols_results2.params - ols_results.params)
            / ols_results.params
            * 100
        ]
    )
)

We can also look at formal statistics for this such as the DFBETAS – a standardized measure of how much each coefficient changes when that observation is left out.

In [26]:
infl = ols_results.get_influence()

In general we may consider DBETAS in absolute value greater than 
 to be influential observations

In [ ]:
2.0 / len(X) ** 0.5

In [ ]:
print(infl.summary_frame().filter(regex="dfb"))

***